In [133]:
import re
import sys
import os
import datetime
sys.path.append('E:\Study\my_github\smart regex')

import smart_regex as sr

# %load_ext line_profiler
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [134]:
patterns = """
#gender | (软?妹 *子|萝 *莉|御 *姐|女 *王|male|female)
#marital_status | (已 *婚|未 *婚|保 *密|(已|未)结 *婚)
#has_children | (已 *育)
#race | ((汉|壮|满|回|苗|维|维 *吾 *尔|土 *家|彝|蒙 *古|蒙)( *族))
#github | ((https?://)?(www.)?github.com/(^| \||;|.|/)+)
#github | ((https?://)?(^| \||;|.|/)+.github.io)
#linkedin | ((https?://)?(www.)?linkedin.com/in/(^| \||;|.|/)+)
$work_year | (应 *届(毕 *业)?生)
#college | ((大 *学|学 *院|学 *校|分 *校|中 *学|一 *中))
#constellation | ((摩 *羯|水 *瓶|双 *鱼|白 *羊|金 *牛|双 *子)座?)
#blood_type | ((o|a|b|ab) *型)
#expect_jnature | (全 *职|兼 *职|实 *习)
#job_cpy_nature | ((民 *营/私 *营)(公 *司|企 *业))
#job_nature | (全 *职|兼 *职|实 *习)
#job_cpy_financing_situation | ((a|b|c|d|e|a+|天 *使|天 *使+)轮)
#job_cpy_financing_situation | ((已|不 *需 *要|无 *需|未)融 *资)
#job_cpy_financing_situation | ((已|非|纳 *斯 *达 *克)上 *市)
#job_cpy_financing_situation | ((h|港|美)股)"""
patterns = [[x.strip() for x in p.split('|', 1)] for p in patterns.split('\n') if p.startswith('#')]

In [135]:
re_patterns = [re.compile(pat[1]) for pat in patterns] * 3

sr_patterns = []
for pat in patterns:
    print('1', pat)
    try:
        sr_patterns.append((pat[0], sr.compile(pat[1])))
    except:
        print(pat)
sr_patterns = sr_patterns * 3

1 ['#gender', '(软?妹 *子|萝 *莉|御 *姐|女 *王|male|female)']
1 ['#marital_status', '(已 *婚|未 *婚|保 *密|(已|未)结 *婚)']
1 ['#has_children', '(已 *育)']
1 ['#race', '((汉|壮|满|回|苗|维|维 *吾 *尔|土 *家|彝|蒙 *古|蒙)( *族))']
1 ['#github', '((https?://)?(www.)?github.com/(^| \\||;|.|/)+)']
1 ['#github', '((https?://)?(^| \\||;|.|/)+.github.io)']
1 ['#linkedin', '((https?://)?(www.)?linkedin.com/in/(^| \\||;|.|/)+)']
1 ['#college', '((大 *学|学 *院|学 *校|分 *校|中 *学|一 *中))']
1 ['#constellation', '((摩 *羯|水 *瓶|双 *鱼|白 *羊|金 *牛|双 *子)座?)']
1 ['#blood_type', '((o|a|b|ab) *型)']
1 ['#expect_jnature', '(全 *职|兼 *职|实 *习)']
1 ['#job_cpy_nature', '((民 *营/私 *营)(公 *司|企 *业))']
1 ['#job_nature', '(全 *职|兼 *职|实 *习)']
1 ['#job_cpy_financing_situation', '((a|b|c|d|e|a+|天 *使|天 *使+)轮)']
1 ['#job_cpy_financing_situation', '((已|不 *需 *要|无 *需|未)融 *资)']
1 ['#job_cpy_financing_situation', '((已|非|纳 *斯 *达 *克)上 *市)']
1 ['#job_cpy_financing_situation', '((h|港|美)股)']


In [113]:
def index_docs(docs=None):
    index = {}
    for doc_idx, doc in enumerate(docs):
        for line_idx, line in enumerate(doc):
            for token in sr.generate_ngram_chars(line, 2):
                if token not in index:
                    index[token] = set()
                index[token].add((doc_idx, line_idx))
    return index

In [114]:
folder = r'E:\Study\my_github\smart regex\test\test_data'
lines = []
for fname in os.listdir(folder):
    if fname.endswith('txt'):
        with open(os.path.join(folder, fname), encoding='utf-8') as f:
            lines.extend(l.strip() for l in f.readlines())
            
folder = r'E:\Study\my_github\smart regex\test\test_data'
docs = []
for fname in os.listdir(folder):
    if fname.endswith('txt'):
        with open(os.path.join(folder, fname), encoding='utf-8') as f:
            docs.append([l.strip() for l in f.readlines()])

In [131]:
%%timeit -n 1 -r 5
# 普通正则匹配时间
re_results = []
for line in lines:
    for re_pat in re_patterns:
        mat = re_pat.search(line)
        if mat:
            re_results.append(mat.group())

212 ms ± 3.51 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [132]:
%%timeit -n 1 -r 5
# smart regex 匹配时间
index = index_docs(docs)

# sr_results_time = {}
sr_results = []
for sr_pat in sr_patterns:
#     now = datetime.datetime.now()
    mat = sr_pat[1].search(docs, index)
    for m in mat:
        sr_results.append(m[2].group())
#     sr_results_time[sr_pat] = (datetime.datetime.now() - now)

54.5 ms ± 2.52 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [118]:
print(len(re_results))
print(len(sr_results))
print(set(re_results)==set(sr_results))

102
102
True


2.25 ms ± 365 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [11]:
sorted(sr_results_time.items(), key=lambda x:x[1], reverse=True)

[(('#polit_status', <data_structs.Expression at 0x1c0a7f6ad30>),
  datetime.timedelta(seconds=1, microseconds=805306)),
 (('#name', <data_structs.Expression at 0x1c0a4dfadd8>),
  datetime.timedelta(microseconds=714093)),
 (('#edu_recruit', <data_structs.Expression at 0x1c0bbdf9470>),
  datetime.timedelta(microseconds=549556)),
 (('#degree', <data_structs.Expression at 0x1c0b2f54748>),
  datetime.timedelta(microseconds=494676)),
 (('#work_year', <data_structs.Expression at 0x1c0ab6daa20>),
  datetime.timedelta(microseconds=198446)),
 (('#job_dept', <data_structs.Expression at 0x1c0bd271860>),
  datetime.timedelta(microseconds=164560)),
 (('#weight', <data_structs.Expression at 0x1c0a5c472e8>),
  datetime.timedelta(microseconds=146608)),
 (('#grad_time', <data_structs.Expression at 0x1c0abcc1438>),
  datetime.timedelta(microseconds=141620)),
 (('#race', <data_structs.Expression at 0x1c0a5d48588>),
  datetime.timedelta(microseconds=140624)),
 (('#edu_time', <data_structs.Expression at 0x1

In [77]:
%prun sr.compile_tokens_to_expression('欧 *阳|独 *孤|令 *狐|皇 *甫|夏 *侯|呼 *延|诸 *葛|完 *颜|拓 *跋|公 *孙|宇 *文|北 *野|欧 *文|上 *官|端 *木')

         10365420 function calls (10263522 primitive calls) in 3.848 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2379    0.578    0.000    2.678    0.001 boolean.py:1287(absorb)
  1489490    0.464    0.000    0.600    0.000 boolean.py:892(__eq__)
1214042/1213655    0.412    0.000    0.596    0.000 boolean.py:728(__eq__)
  4291588    0.403    0.000    0.404    0.000 {built-in method builtins.isinstance}
   269622    0.390    0.000    1.026    0.000 boolean.py:1132(__contains__)
   276616    0.304    0.000    0.591    0.000 boolean.py:935(__init__)
 4059/503    0.279    0.000    3.811    0.008 boolean.py:1142(simplify)
   262640    0.257    0.000    0.822    0.000 boolean.py:1030(__init__)
   370126    0.170    0.000    0.472    0.000 {built-in method builtins.all}
   591619    0.120    0.000    0.153    0.000 boolean.py:941(<genexpr>)
   245480    0.095    0.000    0.120    0.000 boolean.py:1065(cancel)
   164679   

In [108]:
re_results = []
i = 19
for re_pat in re_patterns[i:i+1]:
    for line in lines:
        mat = re_pat.search(line)
        if mat:
            re_results.append(mat.group())
re_results

[]

In [109]:
sr_results = []
for sr_pat in sr_patterns[i:i+1]:
    mat = sr_pat[1].search(docs, index)
    for m in mat:
        sr_results.append(m[2].group())
sr_results

[]

In [79]:
pat = '(https?://)?(www.)?github\.com/(^| \||;|.|/)+'
re_pat = re.compile(pat)
sr_pat = sr.compile(pat)
for line in lines:
    mat = re_pat.search(line)
    if mat:
        print(line)
        print(mat)
        print('\n')
print('========')
print(sr_pat.get_match_query())
mats = sr_pat.search(docs, index)
for m in mats:
    print(m)

github:https://github.com/zhaoxiaoxiao
<re.Match object; span=(7, 38), match='https://github.com/zhaoxiaoxiao'>


.c&b.&co&gi&hu&it&m/&om&th&ub&(( |&.c&/ &b.&co&gi&hu&it&m/&om&th&ub)|(.c&//&b.&co&gi&hu&it&m/&om&th&ub)|(.c&/;&b.&co&gi&hu&it&m/&om&th&ub)|(.c&/^&b.&co&gi&hu&it&m/&om&th&ub))
(0, 29, <re.Match object; span=(7, 38), match='https://github.com/zhaoxiaoxiao'>)
